In [1]:
import numpy as np
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
device

device(type='cuda', index=0)

In [2]:
import torchvision
from torchvision import models
from torchvision import transforms
import os
import glob
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision import models
from random import randint

# tensor -> PIL image
unloader = transforms.ToPILImage()

# flip = transforms.RandomHorizontalFlip(p=1)

In [16]:
class ToyDataset(Dataset):
    def __init__(self, dark_img_dir, light_img_dir):
        self.dark_img_dir = dark_img_dir
        self.light_img_dir = light_img_dir
        self.n_dark = len(os.listdir(self.dark_img_dir))
        self.n_light = len(os.listdir(self.light_img_dir))

    def __len__(self):     
        return min(self.n_dark, self.n_light)

    def __getitem__(self, idx):
        
        filename = os.listdir(self.light_img_dir)[idx]
        
        light_img_path = f"{self.light_img_dir}{filename}"
        light = Image.open(light_img_path).convert("RGB")
        
        dark_img_path = f"{self.dark_img_dir}{filename}"
        dark = Image.open(dark_img_path).convert("RGB")        
        
#         if random()>0.5:
#             light = transforms.functional.rotate(light, 30)
#             dark = transforms.functional.rotate(dark, 30)
#         if random()>0.5:
#             light = transforms.functional.rotate(light, 330)
#             dark = transforms.functional.rotate(dark, 330)
#         if random()>0.5:
#             light = flip(light)
#             dark = flip(dark)

        s = randint(600, 1000)
        transform = transforms.Compose([
            transforms.Resize(s),
            transforms.CenterCrop(600),
            transforms.ToTensor(),
        ])
        
        light = transform(light)
        dark = transform(dark)         

        return dark, light

batch_size = 1
    
train_dark_dir = f"./data/train/dark/"
train_light_dir = f"./data/train/light/"
training_set = ToyDataset(train_dark_dir,train_light_dir)
training_generator = DataLoader(training_set, batch_size=batch_size, shuffle=True)

val_dark_dir = f"./data/test/dark/"
val_light_dir = f"./data/test/light/"
validation_set = ToyDataset(val_dark_dir, val_light_dir)
validation_generator = DataLoader(validation_set, batch_size=batch_size, shuffle=True)

In [4]:
# generate training images
n = 1
cycle = 5
dark_save_path = "./data_augment/train/dark/"
light_save_path = "./data_augment/train/light/"
for i in range(cycle):
    for item in training_generator:
        dark, light = item
        dark = unloader(dark[0,])
        light = unloader(light[0,])
        dark.save(dark_save_path+f"{n}.jpg")
        light.save(light_save_path+f"{n}.jpg")
        n += 1


In [17]:
# generate testing images
n = 1
cycle = 1
dark_save_path = "./data_augment/test/dark/"
light_save_path = "./data_augment/test/light/"
for i in range(cycle):
    for item in validation_generator:
        dark, light = item
        dark = unloader(dark[0,])
        light = unloader(light[0,])
        dark.save(dark_save_path+f"{n}.jpg")
        light.save(light_save_path+f"{n}.jpg")
        n += 1